# medPC wrangler - *two data arrays* 
***
medPC wrangler contains a set of functions to convert medPC output files into a more useable format for further data analysis, as well as pull event counts and latencies that occur within trials or within a certain time window following a given event.

it is *imperative* that the files you are using follow the same structure to the example files. in this case, your file should have 2 data arrays: event codes in array Y and corresponding timestamps in array Z.

the functions are written such that each file is handled as a single session, primarily via the "experiment" label in each subject's header in the file.

the medpc_wrangler.py scripts must also be in the same folder as the files you would like to use.


***

***
to begin using medPC wrangler, first you must import the functions:
***

In [15]:
import medpc_wrangler as med

***
now, set your event codes as they appear in your .mpc code to run your tasks:
***

In [2]:
press = 1 
rwd_delivery = 2
mag_entry = 3 
last_press = 4 
rwd_consumption = 5 
first_press = 6 

***
you likely will want to run multiple files/sessions at once. you can do so using the following loop through each necessary function to create a dataframe as follows:
***

In [3]:
import glob
import pandas as pd

files = glob.glob('/Users/ericatownsend/Desktop/medpc_wrangler/gio_lab_data/*')
# the * will take every file in the folder. change this to match your folder's path.

list_of_session_dfs = []
for file in files: # this will iterate through each datafile for you
    data_strings = med.mpc_to_strings(filename = file)
    cleaned_data = med.extract_data_from_file(file_data = data_strings, multi_array = True)
    time_event_df = med.time_event(data_list = cleaned_data, multi_array = True)
    list_of_session_dfs.append(time_event_df)

# combine all the dataframes for each animal / session to make one big dataframe
final_df = pd.concat(list_of_session_dfs)
final_df['animal_session'] = final_df.session.values + '_' + final_df.subject.values

# make sure everything is numeric
final_df['event'] = pd.to_numeric(final_df['event'])
final_df['time'] = pd.to_numeric(final_df['time'])

the dataframe will appear as so, with subjects, sessions, events, event times parsed out as well as an animal_session column for grouping purposes if you choose to use some other functions:

In [4]:
final_df.head(5)

,subject,session,event,time,animal_session
0,ET1.2_03,CRF6,1.0,20.6,CRF6_ET1.2_03
1,ET1.2_03,CRF6,2.0,20.6,CRF6_ET1.2_03
2,ET1.2_03,CRF6,4.0,20.6,CRF6_ET1.2_03
3,ET1.2_03,CRF6,6.0,20.6,CRF6_ET1.2_03
4,ET1.2_03,CRF6,3.0,32.4,CRF6_ET1.2_03


***
from this point, you can pull out the counts of an event. you can also pull out the counts of an event from another event within a time window (eg, 10 seconds after a cue ends).
***

In [43]:
# counting all magazine entries between the last press of a bout, and the first press of the next bout 
# (in between corresponding event codes)
event_counts_during_trials = med.event_trial_counts(df = final_df, event1 = last_press, event2 = first_press, 
                                                    event_of_interest = mag_entry)

event_counts_during_trials[40:60] # example output

,subject,session,trial,count
40,ET1.2_01,RR2,1,0
41,ET1.2_01,RR2,2,6
42,ET1.2_01,RR2,3,0
43,ET1.2_01,RR2,4,0
44,ET1.2_01,RR2,5,4
45,ET1.2_01,RR2,6,2
46,ET1.2_01,RR2,7,0
47,ET1.2_01,RR2,8,4
48,ET1.2_01,RR2,9,7
49,ET1.2_01,RR2,10,2


***
you can pull out the inter-event latencies in a similar fashion.   
***

In [24]:
# finding average inter-press intervals for each animal and session
inter_press_intervals = med.inter_event_interval(df = final_df, event = press)

inter_press_intervals # example output

,subject,session,average_iei
0,ET1.2_01,CRF6,29.840000
1,ET1.2_03,CRF6,62.415000
2,ET1.2_01,RR2,10.228261
3,ET1.2_03,RR2,30.502500


***
*any of these dataframes can be saved as a csv file using the pandas function* df_name.to_csv('filename.csv')
***